# This is the IBM Data Science Capstone Project

## Import the required Libraries

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ajith_sharma\AppData\Local\Continuum\anaconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    certifi-2018.1.18          |           py36_0         144 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         229 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.19.0-py_0      conda-forge

The following packages will be UPDATED:

    certifi:       2018.1.18-py36_0             --> 2018.1.18-py36_0 conda-forge


Preparing transaction: ...working... done



==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base conda



geopy 1.19.0:            |   0% 
geopy 1.19.0: #######5   |  75% 
geopy 1.19.0: #########7 |  97% 
geopy 1.19.0: ########## | 100% 

certifi 2018.1.18:            |   0% 
certifi 2018.1.18: #######6   |  77% 
certifi 2018.1.18: ########## | 100% 

geographiclib 1.49:            |   0% 
geographiclib 1.49: #######9   |  79% 
geographiclib 1.49: ########## | 100% 


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ajith_sharma\AppData\Local\Continuum\anaconda3

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       hfa6e2cd_0         5.4 MB  conda-forge
    numpy-1.14.2               |   py36h5c71026_0         3.7 MB
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-0.24.2              |   py36h6538335_0         9.4 MB  conda-forge
    blas-1.0                   |              mkl           6 KB
    altair-3.0.1               |           py36_0         732 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  con



==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base conda



openssl 1.0.2r:            |   0% 
openssl 1.0.2r:            |   0% 
openssl 1.0.2r: 7          |   7% 
openssl 1.0.2r: #9         |  19% 
openssl 1.0.2r: ###        |  31% 
openssl 1.0.2r: ####1      |  42% 
openssl 1.0.2r: #####2     |  53% 
openssl 1.0.2r: ######4    |  64% 
openssl 1.0.2r: #######5   |  75% 
openssl 1.0.2r: ########1  |  82% 
openssl 1.0.2r: ########8  |  88% 
openssl 1.0.2r: #########4 |  95% 
openssl 1.0.2r: ########## | 100% 

numpy 1.14.2:            |   0% 
numpy 1.14.2: #          |  10% 
numpy 1.14.2: ##7        |  27% 
numpy 1.14.2: ####4      |  45% 
numpy 1.14.2: ######1    |  62% 
numpy 1.14.2: #######5   |  75% 
numpy 1.14.2: ########3  |  84% 
numpy 1.14.2: #########  |  90% 
numpy 1.14.2: #########5 |  95% 
numpy 1.14.2: #########8 |  99% 
numpy 1.14.2: ########## | 100% 

branca 0

## Data Gathering

In [108]:
## Store the Wikipedia link into a variable
web_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

### Import Beautifulsoup package

In [109]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(web_url,'lxml')


In [110]:
# Scrape data from the website for all the tables
table = soup.find('table', class_="wikitable")
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

#Store in a dataframe
df = pd.DataFrame(res, columns = ["Postcode","Borough","Neighborhood"])
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [111]:
#Check size of initial dataframe
df.shape

(288, 3)

## Data Cleaning and formatting

In [112]:
#Check if there are any "Borough" that is 'Not assigned' and create a new dataframe excluding those rows
df_mod = df[df['Borough']!='Not assigned']

In [113]:
# Print modified dataframe
df_mod.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [114]:
#Check shape of modified data frame
df_mod.shape

(211, 3)

In [115]:
Na_neighborood = df_mod.loc[df_mod['Neighborhood']=="Not assigned"]
Na_neighborood

,Postcode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


In [116]:
df_mod.loc[df_mod['Neighborhood']=="Not assigned",["Neighborhood"]] = df_mod['Borough']

C:\Users\ajith_sharma\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  Ensure that key is valid for current indexer.
C:\Users\ajith_sharma\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [117]:
df_mod.shape

(211, 3)

In [118]:
df_mod = df_mod.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [119]:
df_mod.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [120]:
df_mod.shape

(103, 3)

## Import Geo Location Data from CSV

In [121]:
file = "C:/Users/ajith_sharma/Downloads/IBM Data Science/Capstone/Geospatial_Coordinates.csv"
geo_data = pd.read_csv(file)

# geo_data.head()

In [123]:
#Merge location data with the neighborhood data on post codes
df_location = df_mod.merge(geo_data,how = 'left',left_on = 'Postcode',right_on = 'Postal Code')

In [124]:
df_location.head()

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [127]:
#Drop Postal Code Column
df_location.drop(labels = 'Postal Code',axis =1, inplace = True)

In [128]:
df_location.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [131]:
#Check to make sure all rows still exist
df_location.shape

(103, 5)

## Explore Toronto Data

In [132]:
#Selecting Postcodes which have the word "Toronto" in their Borough and creating a new data frame
df_toronto = df_location[df_location['Borough'].str.contains('Toronto')]

In [133]:
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [135]:
#Check how many rows have Toronto in the name
df_toronto.shape

(38, 5)